In [128]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py


--2024-08-31 19:32:34--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-31 19:32:34 (17.0 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [129]:
import pandas as pd

In [160]:
df = pd.read_csv('../data/data.csv')
df.columns = df.columns.str.lower().str.replace(' ','_')
df = df.fillna("")
df = df.applymap(lambda x: str(x) if isinstance(x, (int)) else x)
df = df.applymap(lambda x: str(int(x*10)) if isinstance(x, (float)) else x)
df.columns

/var/folders/0h/2vn5vvwd7px9f7gshycy68hr0000gp/T/ipykernel_53936/3582601787.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x) if isinstance(x, (int)) else x)
/var/folders/0h/2vn5vvwd7px9f7gshycy68hr0000gp/T/ipykernel_53936/3582601787.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(int(x*10)) if isinstance(x, (float)) else x)


Index(['title', 'year', 'summary', 'short_summary', 'genres', 'imdb_id',
       'runtime', 'youtube_trailer', 'rating', 'movie_poster', 'director',
       'writers', 'cast'],
      dtype='object')

In [240]:
# df.to_csv('../data/data-500.csv', index=False)

In [161]:
# df[10:16]

In [238]:
df = df[:500]
documents = df.to_dict(orient='records')

In [202]:
# df.head()
# documents[10:16]'rating',

In [203]:
import minsearch

In [204]:
index = minsearch.Index(
    text_fields=['title', 'year', 'summary', 'short_summary', 'genres', 'imdb_id',
       'runtime', 'youtube_trailer', 'rating', 'movie_poster', 'director',
       'writers', 'cast'],
    keyword_fields=[]
)

In [205]:
index.fit(documents)

In [206]:
query = "What are the movies of top 10 rating?"

In [207]:
# rt = index.search(query)
# len(rt)
# rt[:3]

In [208]:
prompt_template = """
You provid a user movies' information of our movies rental system.
Formulate 5 questions this user might ask based on a provided movie.
Make the questions specific to this movie.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

title: {title}               
year: {year}     
summary: {summary}     
short_summary: {short_summary}     
genres: {genres}     
imdb_id: {imdb_id}     
runtime: {runtime}     
youtube_trailer: {youtube_trailer}     
rating: {rating}     
movie_poster: {movie_poster}     
director: {director}     
writers: {writers}     
cast: {cast}  

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [209]:
prompt = prompt_template.format(**docs[0])

In [210]:
#prompt

In [211]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [230]:
from openai import OpenAI

# client = OpenAI(
#     base_url='http://localhost:11434/v1/',
#     api_key='ollama',
# )
client = OpenAI()

In [213]:
#llm(prompt)

In [214]:
questions = llm(prompt)


In [215]:
#print(questions)

In [216]:
import json
json.loads(questions)

{'questions': ["What specific personal tragedy does Patton Oswald refer to in his stand-up routine for ''Annihilation''?",
  "How do the themes of Tribulations during Trump era and life after loss manifest as a driving force within 'Annihilation's comedy?",
  "As an artist, how has focusing on such pivotal topics in U.S history affected Patton Oswald’s comedic style or message delivery since the release of ''Annihilation'' according to his interviews and further stand-up material following that year (2017)?",
  "What elements do you incorporate into your acts such as anecdotes, satirical humor etc while addressing sensitive subjects like tribulations during Trump's presidency or public grief in a comedy act? Provide examples from ''Annihilation.'' ",
  "Based on the IMDb rating and runtime of 'Patton Ostrald: Annihilaton', would you say it successfully manages humor while tackling its serious themes, especially for an audience not familiar with Trump's era or personal grief? Why/why no

In [223]:
import os

In [224]:
os.environ['OPENAI_API_KEY'] = '<Your_Key>'

In [231]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [226]:
from tqdm.auto import tqdm

In [227]:
results = {}

In [228]:
len(documents)

500

In [239]:
for doc in tqdm(documents): 
    doc_id = doc['imdb_id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    questions = json.loads(questions_raw)
    results[doc_id] = questions['questions']

  0%|          | 0/500 [00:00<?, ?it/s]

In [241]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [243]:
df_results = pd.DataFrame(final_results, columns=['imdb_id', 'question'])

In [244]:
df_results.to_csv('../data/movies-ground-truth-retrieval-500.csv', index=False)

In [245]:
len(df_results)

2500

In [246]:
documents


[{'title': 'Patton Oswalt: Annihilation',
  'year': '2017',
  'summary': 'Patton Oswald, despite a personal tragedy, produces his best standup yet. Focusing on the tribulations of the Trump era and life after the loss of a loved one, Patton Oswald continues his journey to contribute joy to the world.',
  'short_summary': 'Patton Oswalt, despite a personal tragedy, produces his best standup yet. Focusing on the tribulations of the Trump era and life after the loss of a loved one, Oswalt continues his journey to contribute joy to the world.',
  'genres': 'Uncategorized',
  'imdb_id': 'tt7026230',
  'runtime': '66',
  'youtube_trailer': '4hZi5QaMBFc',
  'rating': '74',
  'movie_poster': 'https://hydramovies.com/wp-content/uploads/2018/04/Patton-Oswalt-Annihilation-Movie-Poster.jpg',
  'director': 'Bobcat Goldthwait',
  'writers': 'Patton Oswalt',
  'cast': 'Patton Oswalt'},
 {'title': 'New York Doll',
  'year': '2005',
  'summary': 'A recovering alcoholic and recently converted Mormon, Ar